# Lab 1 - Logistic Regression with MNIST


# Lab Overview
In this lab we will build and train the Multiclass Logistic Regression model using MNIST data. 

The lab comprises two parts. During the first part, the instructor will walk you through the code to define, train, and evaluate the initial version of MLR model. In the second part you will compete with other students to improve the performance of the model.

The MNIST data consists of hand-written digits with little background noise making it a standard dataset to create, experiment and learn deep learning models with reasonably small comptuing resources.



In [ ]:
from IPython.display import Image
Image(url= "http://3.bp.blogspot.com/_UpN7DfJA0j4/TJtUBWPk0SI/AAAAAAAAABY/oWPMtmqJn3k/s1600/mnist_originals.png", width=200, height=200)


[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) (LR) is a fundamental machine learning technique that uses a linear weighted combination of features and generates probability-based predictions of different classes.  
​
There are two basic forms of LR: **Binary LR** (with a single output that can predict two classes) and **multinomial LR** (with multiple outputs, each of which is used to predict a single class).  

![LR-forms](http://www.cntk.ai/jup/cntk103b_TwoFormsOfLR-v3.png)

In **Binary Logistic Regression** (see top of figure above), the input features are each scaled by an associated weight and summed together.  The sum is passed through a squashing (aka activation) function and generates an output in [0,1].  This output value (which can be thought of as a probability) is then compared with a threshold (such as 0.5) to produce a binary label (0 or 1).  This technique supports only classification problems with two output classes, hence the name binary LR.  In the binary LR example shown above, the [sigmoid][] function is used as the squashing function.

[sigmoid]: https://en.wikipedia.org/wiki/Sigmoid_function

In **Multiclass Linear Regression** (see bottom of figure above), 2 or more output nodes are used, one for each output class to be predicted.  Each summation node uses its own set of weights to scale the input features and sum them together. Instead of passing the summed output of the weighted input features through a sigmoid squashing function, the output is often passed through a `softmax` function (which in addition to squashing, like the `sigmoid`, the `softmax` normalizes each nodes' output value using the sum of all unnormalized nodes). 

$$ h(\textbf{z})_i = \frac{e^{z_i}}{\sum_{k=1}^C{e^{z_k}}} \text{,  where  } \textbf{z} = \textbf{W} \times \textbf{x}  + \textbf{b} $$

In this lab, we will use multinomial LR for classifying the MNIST digits (0-9) using 10 output nodes (1 for each of our output classes).

The figure below summarizes the model in the context of the MNIST data.

![mnist-LR](https://www.cntk.ai/jup/cntk103b_MNIST_LR.png)

During the model training the values of **W** and **b** parameters are optimized to fit the training samples and generalize well to the samples outside of the training set.

This is achieved by iteratively reducing the `loss` function - a function that expresses the "difference" between model predictions and training ground-truth labels. 

`Cross-entropy` is a popular function to measure the loss. It is defined as:

$$ H(\textbf{W},\textbf{b}) = - \sum_{j=1}^C y_j \log (h(\textbf{z})_j ) \text{, where }  y_j \text{ is a ground truth label} $$  

Various optimization approaches can be utilized, Stochastic Gradient Descent (`sgd`) being one of the most popular ones. 

The "classical" stochastic gradient descent uses a single observation to calculate gradient estimates and update the model parameters. This approach is  attractive since it does not require the entire data set (all observation) to be loaded in memory and also requires gradient computation over fewer datapoints, thus allowing for training on large data sets. However, the updates generated using a single observation sample at a time can vary wildly between iterations. An intermediate ground is to use a small set of observations and use an average of the `loss` or error from that set to update the model parameters. This subset is called a *minibatch*.

With minibatches, we often sample observation from the larger training dataset. We repeat the process of model parameters update using different combination of training samples and over a period of time minimize the `loss` (and the error). When the incremental error rates are no longer changing significantly or after a preset number of maximum minibatches to train, we claim that our model is trained.

In the lab we will use the minibatch form of SGD. 




# Code Walkthrough

## Initialize environment

In [ ]:
# Import the relevant components

import sys
import os
import time
import numpy as np
import cntk as C

# Ensure we always get the same amount of randomness
np.random.seed(0)


## Data reading

In this lab we are using the MNIST data pre-processed to follow CNTK CTF format. 


    |labels 0 0 0 0 0 0 0 1 0 0 |features 0 0 0 0 ... 
                                                  (784 integers each representing a pixel)
                                                 

Each line in the file contains two key-value pairs, also refered as streams. The `labels` stream is the one-hot encoded representation of a digit 0-9. The `features` stream is a 784 vector of 0-255 integers representing 28 x 28 pixel grayscale image.

Our dataset includes three files: the training file with 50,000 images, the validation file with 10,000 images, and the testing file with 10,000 images.

To read/sample the files, we define a `create_reader` function that configures and returns the CNTK MinibatchSource object.
    


In [ ]:
training_file = "../Data/MNIST_train.txt"
validation_file = "../Data/MNIST_validate.txt"
test_file = '../Data/MNIST_test.txt'


# Read a CTF formatted text (as mentioned above) using the CTF deserializer from a file
def create_reader(path, is_training, input_dim, num_label_classes):
    labelStream = C.io.StreamDef(field='labels', shape=num_label_classes)
    featureStream = C.io.StreamDef(field='features', shape=input_dim)
    deserializer = C.io.CTFDeserializer(path, C.io.StreamDefs(labels = labelStream, features = featureStream))
    return C.io.MinibatchSource(deserializer,
       randomize = is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)


## Network definition and training
In CNTK, a computational network (e.g. a neural network) is a **Function** object. On one hand a computational network in CNTK is just a function that you can call to apply to data. On the other hand, a computational network contains learnable parameters that can be accessed like object members. Complicated networks can be composed as hierarchies of simpler ones, which, for example, represent layers. 

CNTK function objects are represented internally as graph structures in C++ that encode the computation. This graph structure is wraped in the Python class `Function` that exposes the necessary interface so that other Python functions can call it and access its members (such as learnable parameters)

The **Function** object is CNTK's single abstraction used to represent different operations from simple **basic operations** without learnable parameters to  **layers**, **recurrent step functions**, **complete models**, **criterion functions** and more.


CNTK allows you to create networks using two styles of APIs: Functional API and Graph API.

In this lab we will demonstrate the lower level Graph API. This API is more verbose but sometimes more flexible. In the following labs we will mostly use the Functional API style which is more concise and makes it easier to define complex network topologies.

### Define the network


In [ ]:
def create_mlr_network(input_dim, num_output_classes):
    # Create inputs 
    features = C.input_variable(input_dim)

    # Scale the input features
    feature_scale = 1.0/256.0
    features_norm = C.element_times(feature_scale, features)

    # Define parameters
    W = C.parameter(shape=(input_dim, num_output_classes))
    b = C.parameter(shape=(num_output_classes))

    # And the network
    z = C.times(features_norm, W) + b
    return z


### Create and visualize the network

In [ ]:
input_dim = 784
num_output_classes = 10

z = create_mlr_network(input_dim, num_output_classes)
C.logging.graph.plot(z, "graph.png")
Image("graph.png")

### Configure training
#### Configure a trainer

In [ ]:
def create_trainer(network, labels):
    ## Define loss and metric
    loss = C.cross_entropy_with_softmax(network, labels)
    metric = C.classification_error(network, labels)

    # Create an SGD learner
    lr_schedule = C.learning_rate_schedule(0.2, C.UnitType.minibatch)
    learner = C.sgd(network.parameters, lr_schedule)

    # Create a progress printing helper
    progress_printer = C.logging.ProgressPrinter()

    # Create a trainer
    return C.Trainer(network, (loss, metric), [learner], [progress_printer])


#### Configure a training session

In [ ]:
def train_and_validate(network, training_file, validation_file, mb_schedule, epoch_size, num_epochs):

    # Extract input and output dimensions
    input_dim = network.arguments[0].shape[0]
    num_output_classes = network.outputs[0].shape[0]
    
    # Create the training and validation data set readers
    reader_train = create_reader(training_file, True, input_dim, num_output_classes)
    reader_validate = create_reader(validation_file, False, input_dim, num_output_classes)

     # Define mappings from reader streams to network and ground truth inputs
    features = network.arguments[0]
    labels = C.input_variable(num_output_classes, is_sparse=True)
   
    input_map_training = {
        features: reader_train.streams.features,
        labels: reader_train.streams.labels
    }

    # Define mappings from reader streams to network inputs
    input_map_validation = {
        features: reader_validate.streams.features,
        labels: reader_validate.streams.labels
    }
    
    # Create a trainer
    trainer = create_trainer(network, labels)

    # Set up cross-validation configuration
    cv_config = C.CrossValidationConfig(minibatch_source=reader_validate,
                                        model_inputs_to_streams = input_map_validation,
                                        frequency=None)

    # Create a training session
    training_sess = C.training_session(trainer=trainer,
                                 mb_source=reader_train,
                                 mb_size=mb_schedule,
                                 model_inputs_to_streams=input_map_training,
                                 progress_frequency=epoch_size,
                                 max_samples=epoch_size * num_epochs,
                                 cv_config=cv_config 
                                )

    training_sess.train()

### Execute training

In [ ]:

epoch_size = 6400
num_epochs = 100
mb_schedule = 64

train_and_validate(z, training_file, validation_file, mb_schedule, epoch_size, num_epochs)

## Inference

    The trained network can be used like a "normal Python function" to score new data. Since CNTK supports `numpy` interoperability you can pass `numpy` arrays as input data

In [ ]:
# Read some data for scoring

reader = create_reader(test_file, False,  input_dim, num_output_classes)

input_map = {
        input: reader.streams.features
    }

data = reader.next_minibatch(10, input_map = input_map)

img_data = data[input].asarray()


In [ ]:
# Add softmax node and call the network on an image tensor
out = C.softmax(z)
pred = out(img_data[0])

print(pred)
print(np.argmax(pred))

# Hackathon

Try to improve the performance of the model. 

Hints:
- Play with the learning rate, minibatch size and the number of epochs
- You can look at regularization - check `l1_regularization` and `l2_regularization` hyper parameters of the `sgd` learner

## Final testing


DON'T CHEAT. DON'T USE MNIST_test.txt FOR MODEL TRAINING AND SELECTION. DON'T EXECUTE THE BELOW CELL TILL YOU ARE READY FOR THE FINAL TEST


In [ ]:
def final_evaluation(network, test_file):
    
    # Extract input and output dimensions
    input_dim = network.arguments[0].shape[0]
    num_output_classes = network.outputs[0].shape[0]
    
    # Create the test data set readers
    reader = create_reader(test_file, False,  input_dim, num_output_classes)

     # Define mappings from reader streams to network and ground truth inputs
    features = network.arguments[0]
    labels = C.input_variable(num_output_classes, is_sparse=True)
   
    input_map = {
        features: reader.streams.features,
        labels: reader.streams.labels
    }
   
    metric = C.classification_error(network, labels)
    
    evaluator = C.Evaluator(metric, [C.logging.ProgressPrinter()])
    
    minibatch_size = 1024
    data = reader.next_minibatch(minibatch_size, input_map=input_map)
    while bool(data):
        evaluator.test_minibatch(data)
        data = reader.next_minibatch(minibatch_size, input_map=input_map)
    evaluator.summarize_test_progress()
    

In [ ]:


final_evaluation(z, test_file)